In [3]:
pip install tenseal


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 33.7 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import tenseal as ts


df = pd.read_csv("diabetes.csv")


X = df.drop('Outcome', axis=1)
y = df[['Outcome']]


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


print("Starting SVM Linear")
svc_linear = SVC(kernel='linear')
svc_linear.fit(X_train, y_train.values.ravel())
print("SVM Linear Completed")

print("Starting SVM Polynomial")
svc_poly = SVC(kernel='poly', degree=3, gamma=2)
svc_poly.fit(X_train, y_train.values.ravel())
print("SVM Poly Completed")

# Save Results
decision_function = svc_linear.decision_function(X_test)
ytestscore = decision_function[0]

decision_function_poly = svc_poly.decision_function(X_test)
ytestscore_poly = decision_function_poly[0]

import os


os.makedirs("models", exist_ok=True)
os.makedirs("data", exist_ok=True)

# Save model outputs and parameters
np.savetxt("models/weights.txt", svc_linear.coef_)
np.savetxt("models/intercept.txt", svc_linear.intercept_)
np.savetxt("data/ytestscore.txt", [ytestscore])

np.savetxt("models/dual_coef.txt", svc_poly.dual_coef_)
np.savetxt("models/support_vectors.txt", svc_poly.support_vectors_)
np.savetxt("models/intercept_poly.txt", svc_poly.intercept_)

print("---- Model Training Completed! ----")

# CKKS context for Homomorphic Encryption (TenSEAL)
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2**40
context.generate_galois_keys()

x = X_test[0]
weights = svc_linear.coef_[0]
intercept = svc_linear.intercept_[0]


enc_x = ts.ckks_vector(context, x)
enc_weights = ts.ckks_vector(context, weights)
enc_intercept = ts.ckks_vector(context, [intercept])


enc_result = enc_x.dot(enc_weights) + enc_intercept


decrypted_result = enc_result.decrypt()


decrypted_value = decrypted_result[0] if isinstance(decrypted_result, (list, np.ndarray)) else decrypted_result


print("Homomorphic Encrypted SVM Score (Decrypted):", decrypted_value)


prediction = 1 if decrypted_value > 0 else 0
print("Predicted Class (Diabetic = 1, Non-Diabetic = 0):", prediction)


from sklearn.metrics import accuracy_score


y_pred_linear = svc_linear.predict(X_test)
acc_linear = accuracy_score(y_test, y_pred_linear)
print("Linear SVM Accuracy:", acc_linear)


y_pred_poly = svc_poly.predict(X_test)
acc_poly = accuracy_score(y_test, y_pred_poly)
print("Polynomial SVM Accuracy:", acc_poly)


Starting SVM Linear
SVM Linear Completed
Starting SVM Polynomial
SVM Poly Completed
---- Model Training Completed! ----
Homomorphic Encrypted SVM Score (Decrypted): -0.8145359058975753
Predicted Class (Diabetic = 1, Non-Diabetic = 0): 0
Linear SVM Accuracy: 0.7597402597402597
Polynomial SVM Accuracy: 0.7142857142857143


In [9]:

feature_names = [
    "Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
    "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"
]

print("\n Enter the following health information:")


user_input = []
for feature in feature_names:
    while True:
        try:
            val = float(input(f"{feature}: "))
            if val < 0:
                print(" Please enter a non-negative value.")
                continue
            user_input.append(val)
            break
        except ValueError:
            print(" Invalid input. Please enter a number.")


user_input_scaled = scaler.transform(user_input_df)

# Encrypt the scaled input
enc_user_x = ts.ckks_vector(context, user_input_scaled[0])
enc_weights_linear = ts.ckks_vector(context, svc_linear.coef_[0])
enc_intercept_linear = ts.ckks_vector(context, [svc_linear.intercept_[0]])


enc_result_linear = enc_user_x.dot(enc_weights_linear) + enc_intercept_linear
decrypted_linear = enc_result_linear.decrypt()[0]
prediction_linear = 1 if decrypted_linear > 0 else 0


prediction_poly = svc_poly.predict(user_input_scaled)[0]


print("\n HOMOMORPHIC ENCRYPTED PREDICTION (Linear SVM)")
print("--------------------------------------------------")
print(f" Score = {decrypted_linear:.4f}  Prediction = {prediction_linear} ({'Diabetic' if prediction_linear == 1 else 'Non-Diabetic'})")

print("\n Regular Prediction (Polynomial SVM)")
print("--------------------------------------------------")
print(f" Prediction = {prediction_poly} ({'Diabetic' if prediction_poly == 1 else 'Non-Diabetic'})")



 Enter the following health information:
Pregnancies: 6
Glucose: 148
BloodPressure: 72
SkinThickness: 35
Insulin: 0
BMI: 33.6
DiabetesPedigreeFunction: 0.627
Age: 50

 HOMOMORPHIC ENCRYPTED PREDICTION (Linear SVM)
--------------------------------------------------
 Score = 0.9254  Prediction = 1 (Diabetic)

 Regular Prediction (Polynomial SVM)
--------------------------------------------------
 Prediction = 0 (Non-Diabetic)


In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
